In [1]:
from data.helper import *

In [2]:
DF = get_plan_df()

100%|██████████| 10800/10800 [00:00<00:00, 201263.08it/s]


In [ ]:
for domain in DOMAINS:
    df = DF.copy()
    print(domain)
    df = df[df['domain'] == domain]
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(['pruning', 'optimiser', 'iterations']).sum().reset_index()
    df = df[['pruning', 'optimiser', 'iterations', 'solved']]
    # pivot table
    df = df.pivot_table(index=['pruning', 'optimiser'], columns='iterations', values='solved')
    # make entries int2
    df = df.astype(int)
    df = df.fillna(0)
    display(df)

blocksworld


iterations           1   2   3   4   5   6
pruning  optimiser                        
collapse rank-lp     0   0   0   0   0   0
         rank-svm    0   0   0   0   0   0
         svr         0   0   0   0   0   0
none     rank-lp     4  11  22   1   0   0
         rank-svm    4   3  13   8   0   0
         svr        16  12  21  20  20  17

childsnack


iterations          1  2  3  4  5  6
pruning  optimiser                  
collapse rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0
none     rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0

ferry


...

iterations          1  2  3  4  5  6
pruning  optimiser                  
collapse rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0
none     rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0

spanner


iterations          1  2  3  4  5  6
pruning  optimiser                  
collapse rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0
none     rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0

transport


iterations          1  2  3  4  5  6
pruning  optimiser                  
collapse rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0
none     rank-lp    0  0  0  0  0  0
         rank-svm   0  0  0  0  0  0
         svr        0  0  0  0  0  0